# Going for the pre-trained network

In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image
print('test run')

test run


## Get the pre-trained model

In [16]:
rs50 = torchvision.models.resnet50(pretrained=True)
print(rs50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Keep only the first layers

In [18]:
# keep the first layers of net
modules=list(rs50.children())[:4]
convNet = nn.Sequential(*modules)

# fix the parameters
for p in convNet.parameters():
    p.requires_grad=False
    
print(convNet)

# keep just the first layer 
mods=list(rs50.children())[:1]
conv = nn.Sequential(*mods)

# fix the parameters
for p in conv.parameters():
    p.requires_grad=False
    
print(conv)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)
Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
)


## Get Image Features

The output is a 64 channel tensor ( x W x H) that can be sent to our network

In [19]:
transform = transforms.Compose([transforms.ToTensor()])
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(imgName):
    im = Image.open(imgName)
    im = transform(im)
    im = torch.unsqueeze(im,0)
    return im#.cuda()

dataDir = '/mindhive/nklab5/users/hlk/packages/RAFT/demo-frames/'     
i = 16
imPath = '{0}frame_00{1}.png'.format(dataDir,i)
im = image_loader(imPath)                           

print('image')
print(np.shape(im))
print('features 4 layers')
f = convNet(im)
f.size()
print('features 1 layer')
f2 = conv(im)
f2.size()

image
torch.Size([1, 3, 436, 1024])
features 4 layers
features 1 layer


torch.Size([1, 64, 218, 512])

## Save Image Features